<a href="https://colab.research.google.com/github/MichaelKoinakis/StarWarsChatBot/blob/main/ChatWithChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import necessities
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [ ]:
device

device(type='cpu')

In [ ]:
print(torch.cuda.device_count())

0


In [ ]:
# print(torch.cuda.get_device_name(0))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# To delete
corpus_name = "star_wars"
# corpus = os.path.join("/content/drive/My Drive/data", corpus_name)
filesList = os.listdir('/content/drive/My Drive/StarWarsBotData')

In [ ]:
containerList = []
for file in filesList:
  importData = open('/content/drive/My Drive/StarWarsBotData'+'/'+file, 'r', encoding='UTF-8', errors='ignore')
  df = importData.readlines()
  importData.close()
  for data in df:
    _data = data.split('"')[-2]
    containerList.append(_data)

In [ ]:
hugeContainerList = []
for i in range(2, 11):
  for j in range(len(containerList) - i):
    hugeContainerList.append(containerList[j:j+i+1])

In [ ]:
# Define path to new file
datafile = '/content/drive/My Drive/formattedStarWarsLines.txt'

delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

In [ ]:
# Write new csv file
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter)
    for pair in hugeContainerList:
        writer.writerow(pair)


Writing newly formatted file...


In [ ]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [ ]:
MAX_LENGTH = 30  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

In [ ]:
# Lowercase, trim, and remove non-letter characters
# Lowercase, trim, and remove non-letter characters
def normalizeString(text):
    text = unicodeToAscii(text.lower().strip())
    
    text = re.sub(r"[\(\[].*?[\)\]]", "", text)
    
    text = re.sub(r"a'ight", "alright", text)
    
    text = re.sub(r"ain't", "are not", text) # or am not/ is not/ are not/ has not/ have not/ did not
    text = re.sub(r"amn't", "am not", text)
    text = re.sub(r"arencha", "are you not", text)
    text = re.sub(r"aren't", "are not", text)
    
    text = re.sub(r"'bout'", "about", text)
    
    text = re.sub(r"can not", "cannot", text)
    text = re.sub(r"can't", "cannot", text)
    
    text = re.sub(r"cap'n", "captain", text)
    
    text = re.sub(r"'cause'", "because", text)
    
    text = re.sub(r"'cept'", "except", text)
    
    text = re.sub(r"could've", "could have", text)
    text = re.sub(r"couldn't", "could not", text)
    text = re.sub(r"couldn't've", "could not have", text)
    
    text = re.sub(r"dammit", "damn it", text)
    text = re.sub(r"daren't", "dare not", text)
    
    text = re.sub(r"didn't", "did not", text)
    text = re.sub(r"doesn't", "does not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"dunno", "do not know", text) # or does not
    
    text = re.sub(r"d'ye", "do you", text) # or did you
    text = re.sub(r"e'en", "even", text)
    text = re.sub(r"e'er", "ever", text)
    text = re.sub(r"'em", "them", text)
    
    text = re.sub(r"everybody's", "everybody is", text)
    text = re.sub(r"everyone's", "everyone is", text)
    
    text = re.sub(r"fo'c'sle", "forecastle", text)
    
    text = re.sub(r"'gainst'", "against", text)
    
    text = re.sub(r"g'day", "good day", text)
    text = re.sub(r"gimme", "give me", text)
    text = re.sub(r"giv'n", "given", text)
    text = re.sub(r"gonna", "going to", text)
    text = re.sub(r"gon't", "go not", text)
    
    text = re.sub(r"hadn't", "had not", text)
    text = re.sub(r"had've", "had have", text)
    text = re.sub(r"hasn't", "has not", text)
    text = re.sub(r"haven't", "have not", text)
    text = re.sub(r"he'd", "he had", text) # or he would
    text = re.sub(r"he'll", "he will", text) # or he shall
    text = re.sub(r"helluva", "hell of a", text)
    text = re.sub(r"he's", "he is", text) # or he has
    
    text = re.sub(r"here's", "here is", text)
    
    text = re.sub(r"how'd", "how did", text) # or how would
    text = re.sub(r"howdy", "how do you do", text) # or how do you fare
    text = re.sub(r"how'll", "how will", text)
    text = re.sub(r"how're", "how are", text)
    text = re.sub(r"how's", "how is", text) # or how has/ how does
    
    text = re.sub(r"i'd've", "i would have", text)
    text = re.sub(r"i'd", "i had", text) # or i would
    text = re.sub(r"i'll", "i will", text) # or i shall
    text = re.sub(r"i'm", "i am", text)
    
    text = re.sub(r"imma", "i am about to go", text)
    
    text = re.sub(r"i'm'o", "i am going to", text)
    
    text = re.sub(r"innit", "is it not", text)
    
    text = re.sub(r"ion", "i do not", text)
    
    text = re.sub(r"i've", "i have", text)
    
    text = re.sub(r"isn't", "is not", text)
    
    text = re.sub(r"it'd", "it would", text)
    text = re.sub(r"it'll", "it will", text) # or it shall
    text = re.sub(r"it's", "it is", text) # or it has
    
    text = re.sub(r"iunno", "i do not know", text)
    
    text = re.sub(r"kinda", "kind of", text)
    
    text = re.sub(r"let's", "let us", text)
    
    text = re.sub(r"ma'am", "madam", text)
    
    text = re.sub(r"mayn't", "may not", text)
    text = re.sub(r"may've", "may have", text)
    
    text = re.sub(r"methinks", "i think", text)
    
    text = re.sub(r"mightn't", "might not", text)
    text = re.sub(r"might've", "might have", text)
    
    text = re.sub(r"mustn't've", "must not have", text)
    text = re.sub(r"mustn't", "must not", text)
    text = re.sub(r"must've", "must have", text)
    
    text = re.sub(r"'neath'", "beneath", text)
    text = re.sub(r"needn't", "need not", text)
    
    text = re.sub(r"nal", "and all", text)
    text = re.sub(r"ne'er", "never", text)
    
    text = re.sub(r"o'clock", "of the clock", text)
    text = re.sub(r"o'er", "over", text)
    text = re.sub(r"ol'", "old", text)
    text = re.sub(r"oughtn't", "ought not", text)
    text = re.sub(r"'round", "around", text)
    
    text = re.sub(r"shalln't", "shall not", text)
    text = re.sub(r"shan't", "shall not", text)
   
    text = re.sub(r"she'd", "she had", text) # or she would
    text = re.sub(r"she'll", "she will", text) # or she shall
    text = re.sub(r"she's", "she is", text) # or she has
    
    text = re.sub(r"shouldn't've", "should not have", text)
    text = re.sub(r"should've", "should have", text)
    text = re.sub(r"shouldn't", "should not", text)
    
    text = re.sub(r"somebody's", "somebody has", text) # or somebody is
    text = re.sub(r"someone's", "someone has", text) # or someone is
    text = re.sub(r"something's", "something has", text) # or something is
    
    text = re.sub(r"so're", "so are", text) # so are
    text = re.sub(r"so's", "so is", text) # so is
    text = re.sub(r"so've", "so have", text)
    
    text = re.sub(r"that'll", "that will", text) # or that shall
    text = re.sub(r"that're", "that are", text)
    text = re.sub(r"that's", "that is", text) # or that has
    text = re.sub(r"that'd", "that had", text) # or that would
    
    text = re.sub(r"there'd", "there had", text) # or there would
    text = re.sub(r"there'll", "there will", text) # or there shall
    text = re.sub(r"there're", "there are", text)
    text = re.sub(r"there's", "there is", text) # or there has
    text = re.sub(r"there're", "these are", text)
    
    text = re.sub(r"these've", "these have", text)
    
    text = re.sub(r"they'd", "they had", text) # or they would
    text = re.sub(r"they'll", "they will", text) # or they shall
    text = re.sub(r"they're", "they are", text) # or they were
    text = re.sub(r"they've", "they have", text)
    text = re.sub(r"this's", "this is", text) # or this has
    text = re.sub(r"those're", "those are", text)
    text = re.sub(r"those've", "those have", text)
    
    text = re.sub(r"'thout", "without", text)
    text = re.sub(r"'till'", "untill", text)
    text = re.sub(r"'tis'", "it is", text)
    text = re.sub(r"to've", "to have", text)
    text = re.sub(r"'twas'", "it was", text)
    text = re.sub(r"'tween'", "between", text)
    text = re.sub(r"'twere'", "it were", text)
    
    text = re.sub(r"wanna", "want to", text)
    text = re.sub(r"wasn't", "was not", text)
    text = re.sub(r"we'd've", "we would have", text)
    text = re.sub(r"we'd", "we had", text) # or we would, we did
    text = re.sub(r"we'll", "we will", text) # or we shall
    text = re.sub(r"we're", "we are", text)
    text = re.sub(r"we've", "we have", text)
    text = re.sub(r"weren't", "were not", text)
    
    text = re.sub(r"whatcha", "what are you", text) # or what about you
    
    text = re.sub(r"what'd", "what did", text)
    text = re.sub(r"what'll", "what will", text) # or what shall
    text = re.sub(r"what're", "what are", text) # or what were
    text = re.sub(r"what's", "what is", text) # or what has / what is / what does
    text = re.sub(r"what've", "what have", text)
    
    text = re.sub(r"when's", "when is", text) # or when has
    
    text = re.sub(r"where'd", "where did", text)
    text = re.sub(r"where'll", "where will", text) # or where shall
    text = re.sub(r"where're", "where are", text)
    text = re.sub(r"where's", "where is", text) # or where has
    text = re.sub(r"where've", "where have", text)
    
    text = re.sub(r"which'd", "which had", text) # or which would
    text = re.sub(r"which'll", "which will", text) # or which shall
    text = re.sub(r"which're", "which are", text)
    text = re.sub(r"which's", "which is", text) # or which has
    text = re.sub(r"which've", "which have", text)

    text = re.sub(r"who'd've", "who would have", text)
    text = re.sub(r"who'd", "who had", text) # or who would/ who did
    text = re.sub(r"who'll", "who will", text) # or who shall
    text = re.sub(r"who're", "who are", text)
    text = re.sub(r"who's", "who is", text) # or who has
    text = re.sub(r"who've", "who have", text)

    text = re.sub(r"why'd", "why did", text)
    text = re.sub(r"why're", "why are", text)
    text = re.sub(r"why's", "why is", text) # or why has / why does
    
    text = re.sub(r"willn't", "will not", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"wonnot", "will not", text)

    text = re.sub(r"wouldn't've", "would not have", text)
    text = re.sub(r"wouldn't", "would not", text)
    text = re.sub(r"would've", "would have", text)

    text = re.sub(r"y'all're", "you all are", text)
    text = re.sub(r"y'all'd'n't've", "you all would not have", text)
    text = re.sub(r"y'all'd've", "you all would have", text)
    text = re.sub(r"y'all", "you all", text)
    text = re.sub(r"y'at", "you at", text)
    
    text = re.sub(r"yes'm", "yes madam", text)
    text = re.sub(r"yessir", "yes sir", text)
    text = re.sub(r"yesn't", "yes not", text)
    
    text = re.sub(r"you'd", "you had", text) # or you would
    text = re.sub(r"you'll", "you will", text) # or you shall
    text = re.sub(r"you're", "you are", text)
    text = re.sub(r"you've", "you have", text)

    text = re.sub(r"'s", "has", text) # or is/ does/ us
    text = re.sub(r"'re", "were", text)
    text = re.sub(r"'ve'", "have", text)
    text = re.sub(r"'d", "would", text)

    text = re.sub(r"\.+", ".", text)
    
    text = re.sub(r"[-()\"#/@;:<>{}+=~|,]\n", "", text)
    
    return text

In [ ]:
# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

In [ ]:
# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

In [ ]:
# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs

In [ ]:
# Load/Assemble voc and pairs
save_dir = '/content/drive/My Drive/'
voc, pairs = loadPrepareData(corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 346221 sentence pairs
Trimmed to 337428 sentence pairs
Counting words...
Counted words: 20227

pairs:
[' captain.', 'yes, sir?', ' tell them we wish to board at once.']
['yes, sir?', ' tell them we wish to board at once.', 'yes, sir.']
[' tell them we wish to board at once.', 'yes, sir.', ' with all due respect for the trade federati do not, the ambassodors for the supreme chancellor wish to board immediately.']
['yes, sir.', ' with all due respect for the trade federati do not, the ambassodors for the supreme chancellor wish to board immediately.', 'yes, yes, of coarse.ahhh.as you know, our blockade is perfectly legal, and we had be happy to recieve the ambassador.happy to.']
[' with all due respect for the trade federati do not, the ambassodors for the supreme chancellor wish to board immediately.', 'yes, yes, of coarse.ahhh.as you know, our blockade is perfectly legal, and we had be happy to recieve the ambassador.happy to.', '

In [ ]:
MIN_COUNT = 5    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 20222 / 20224 = 0.9999
Trimmed from 337428 pairs to 337425, 1.0000 of total


In [ ]:
# From this point everything remains the same.
pairs[:20]

[[' captain.', 'yes, sir?', ' tell them we wish to board at once.'],
 ['yes, sir?', ' tell them we wish to board at once.', 'yes, sir.'],
 [' tell them we wish to board at once.',
  'yes, sir.',
  ' with all due respect for the trade federati do not, the ambassodors for the supreme chancellor wish to board immediately.'],
 ['yes, sir.',
  ' with all due respect for the trade federati do not, the ambassodors for the supreme chancellor wish to board immediately.',
  'yes, yes, of coarse.ahhh.as you know, our blockade is perfectly legal, and we had be happy to recieve the ambassador.happy to.'],
 [' with all due respect for the trade federati do not, the ambassodors for the supreme chancellor wish to board immediately.',
  'yes, yes, of coarse.ahhh.as you know, our blockade is perfectly legal, and we had be happy to recieve the ambassador.happy to.',
  'they must be important if the viceroy sent one of those useless protocol gearheads to greet them.'],
 ['yes, yes, of coarse.ahhh.as you k

In [ ]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

In [ ]:
def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

In [ ]:
def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

In [ ]:
# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

In [ ]:
# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

In [ ]:
# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len

In [ ]:
# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

In [ ]:
print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[  124,   448,    36,  2393, 14778],
        [  126,   651,    70,     2,     2],
        [   21,    62, 12398,     0,     0],
        [ 3113,  2732, 12399,     0,     0],
        [ 6117,    24,     2,     0,     0],
        [    9,   173,     0,     0,     0],
        [ 1430,     2,     0,     0,     0],
        [  230,     0,     0,     0,     0],
        [ 3683,     0,     0,     0,     0],
        [    2,     0,     0,     0,     0]])
lengths: tensor([10,  7,  5,  2,  2])
target_variable: tensor([[    9,   105,    70,   166, 11839],
        [   86,    36,    36,  4608, 11840],
        [  147,    62,  2664,   771,  4166],
        [  271,  2732,     2,     2,     2],
        [ 4635,    24,     0,     0,     0],
        [    2,   315,     0,     0,     0],
        [    0,     2,     0,     0,     0]])
mask: tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0],
      

In [ ]:
# The Encoder

class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

In [ ]:
# The Luong attention layer

class Attn(torch.nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = torch.nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = torch.nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = torch.nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [ ]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

In [ ]:
# Masked Loss

def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [ ]:
# The training process

def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [ ]:
# Training iterations

def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Possible error issues below, due to the os.path
        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [ ]:
# The GreedySearchDecoder

class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros(0, dtype=torch.long)
        all_scores = torch.zeros(0)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [ ]:
# Text evaluation step

def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [ ]:
# Model...Assemble!
# Configure models
model_name = 'star_wars_model'
#attn_model = 'dot'
attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 8
decoder_n_layers = 8
dropout = 0.1
batch_size = 128

# Set checkpoint to load from; set to None if starting from scratch
# loadFilename = None
checkpoint_iter = 16000
loadFilename = '/content/drive/My Drive/star_wars_model/star_wars/8-8_500/attention_general.tar'


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    # checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')


Building encoder and decoder ...
Models built and ready to go!


In [ ]:
# Model Training

# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 16000
print_every = 500
save_every = 16000

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# Run training iterations
# print("Starting Training!")
# trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
#           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
#           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...


In [ ]:
# Evaluate decoder and encoder
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

In [ ]:
# Implementation
# Begin chatting (uncomment and run the following line to begin)
evaluateInput(encoder, decoder, searcher, voc)

> ηο
Error: Encountered unknown word.
> q
